In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 110%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
import numpy as np
import pandas as pd

from get_data.hh_requests import vacancy_keyword_check, update_vacancies_description

df = pd.DataFrame(columns=["id", "under consideration", "checked", "have full descript", "description", "nonvalid vacancy", "telegram moder", "moderated date"])
df = vacancy_keyword_check("Motion designer", df, period=5)
df = update_vacancies_description(df)
df.head(5)

# saving data backup TODO:add date and profession to file name
df[["description","snippet.requirement","snippet.responsibility"]].to_csv("description.csv")

/home/ambisinistra/miniconda3/envs/HHbot/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [3]:
df.head()

,id,under consideration,checked,have full descript,description,nonvalid vacancy,telegram moder,moderated date,premium,name,...,address.id,address.metro,employer.logo_urls,contacts.name,contacts.email,contacts.phones,department.id,department.name,insider_interview.id,insider_interview.url
0,38793634,0,0,1,Arival is an early-stage digital banking start...,NaN,NaN,NaN,False,Design Director,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,38865330,0,0,1,\n\n **Apalon** \- лидер среди компаний-разраб...,NaN,NaN,NaN,True,Motion Designer,...,618054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,38376305,0,0,1,\n\n**О наc** :\n\nOrganicSoft является Резиде...,NaN,NaN,NaN,False,Motion Designer,...,3678228,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,38875096,0,0,1,"\n\nВ команду креативных людей, которые создаю...",NaN,NaN,NaN,False,Motion Designer,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,38685846,0,0,1,\n\nЕсли Вы талантливый и трудолюбивый Motion ...,NaN,NaN,NaN,False,Motion Designer (офис м. Автозаводская),...,554012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from text_processing.segmentation_by_terminals import segmentation_by_terminals, return_first_items

def build_vacancy_text(number_in_db : int, df : pd.DataFrame):
    cell = df.iloc[number_in_db]

    vacancy_text = ""
    vacancy_text += cell["name"] + "\n"
    vacancy_text += cell["employer.name"] + "\n"
    vacancy_text += cell["area.name"] + "\n\n"
    
    description_segments = return_first_items(segmentation_by_terminals(cell["description"], {}))
    
    for header in description_segments.keys():
        vacancy_text += header + "\n\n"
        for paragraph_string in description_segments[header]:
            vacancy_text += "- " + paragraph_string + "\n"
        vacancy_text += "\n"
    
    vacancy_text += "Страница компании" + "\n"
    vacancy_text += cell["employer.alternate_url"] + "\n\n"
    vacancy_text += "Страница вакансии" + "\n"
    vacancy_text += cell["alternate_url"]

    return vacancy_text

In [5]:
vacancy = build_vacancy_text(0, df)

print(vacancy)

Design Director
Arival
Москва

EVERYDAY TASKS:

- Operate as Arival Bank’’s lead brand designer, ensuring that all iterations of the brand maintain a close connection to our design style guide and that we maintain a consistent design voice across every platform and consumer touchpoint, from stills to motion
- Ensure design deliverables for internal and external usage are delivered to key stakeholders with a high level of accuracy, industry-leading design solutions, and best-in-class execution
- Help design and execute multi platform campaigns for Arival’s brand marketing initiatives, including custom shoot content, toolkits, logos, packaging, key art, etc.
- Work with product & marketing partners on design needs/assets
- Coordinate with our internal teams on initiatives to create cultural currency and conversation, and drive customer acquisition, engagement, and retention
- Help maintain and update our brand style guide and toolkit
- Help design and create assets for stunts and events


In [10]:
from my_telebot_utils import * #make_moder_keyboard, make_post_keyboard, make_suspicious_keyboard, make_irrelevant_keyboard, make_edit_keyboard
#from telebot.types import InlineKeyboardMarkup
#from telebot.types import InlineKeyboardButton 

from token import get_token
import telebot

TOKEN = get_token()
bot = telebot.TeleBot(token=TOKEN, threaded=False)
counter = 0

# When moderator ask for vacancy bot should check for unnocuppied one,
# mark it and give to moderator

@bot.message_handler(commands=["check"])
def check(message):
    keyboard = make_moder_keyboard()
    text = build_vacancy_text(0, df)
    df["under consideration"].iloc[0] = 1
    df["telegram moder"].iloc[0] = message.from_user.id
    
    # counter += 1
    bot.send_message(message.chat.id, text=text, reply_markup=keyboard)
    #print (message.chat.id)

@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    index = df[df["telegram moder"] == call.from_user.id].index #preassume there only one row
    #recieving what key on message keyboard was pressed and editing keyboard accordingly
    if call.data == "post":
        keyboard = make_post_keyboard()
        bot.answer_callback_query(call.id, "vacancy was posted")
    elif call.data == "irre":
        keyboard = make_irrelevant_keyboard()
        bot.answer_callback_query(call.id, "vacancy was marked as irrelevant")
    elif call.data == "susp":
        keyboard = make_irrelevant_keyboard()
        bot.answer_callback_query(call.id, "vacancy was marked as suspicious")
    elif call.data == "othe":
        keyboard = make_irrelevant_keyboard()
        bot.answer_callback_query(call.id, "vacancy was marked as out of a box")
    elif call.data == "edit":
        keyboard = make_edit_keyboard()
        bot.answer_callback_query(call.id, "please send me editeded vacancy post")
    
    # depend on type of callback editing message keyboard. In case of edit button change text message also  
    bot.edit_message_text(text=call.message.text, chat_id = call.from_user.id, message_id=call.message.message_id, reply_markup=keyboard)
    
bot.polling()

/home/ambisinistra/miniconda3/envs/HHbot/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
